### langchain 메소드's

-  1. `stuff`
	- **방식:** 모든 데이터를 한 번에 처리.
	- **장점:** 단순, 빠름.
	- **단점:** 토큰 제한에 취약.
	- **사용:** 소규모 데이터 작업.

-  2. `map_reduce`
	- **방식:** 데이터를 나누어(Map) 처리 후 결과 결합(Reduce).
	- **장점:** 대규모 데이터 처리 가능.
	- **단점:** 결합 과정에서 정보 손실 위험.
	- **사용:** 대량 문서 요약, 복잡한 쿼리.

-  3. `map_rerank`
	- **방식:** 데이터를 나누어(Map) 처리하고 중요도별로 정렬.
	- **장점:** 가장 관련성 높은 결과 제공.
	- **단점:** 정렬 기준의 정확도에 의존.
	- **사용:** 질문과 관련된 문서 부분 찾기.

-  4. `refine`
	- **방식:** 초기 결과를 생성하고 단계적으로 개선.
	- **장점:** 높은 품질 결과.
	- **단점:** 느리고 계산 비용 높음.
	- **사용:** 긴 문서 요약, 세부적인 응답 생성.

-  5. `combine_map`
	- **방식:** 여러 문서를 개별 처리 후 통합.
	- **장점:** 각 문서에서 핵심 정보 추출 후 결합.
	- **사용:** 분산된 정보를 하나로 정리.

-  6. `qa_over_docs`
	- **방식:** 문서 내에서 질문에 대한 답변 찾기.
	- **장점:** 특정 정보 검색에 최적화.
	- **사용:** faq 생성, 특정 질문 응답.

-  7. `iterative_summarization`
	- **방식:** 데이터를 반복적으로 요약하며 축소.
	- **장점:** 매우 큰 문서 처리 가능.
	- **사용:** 대규모 문서 요약.

-  8. `split_and_answer`
	- **방식:** 문서를 청크로 나누고 각 청크에서 답변 생성.
	- **장점:** 섹션별 독립적 응답 제공.
	- **사용:** 구조화된 문서에서 질문 응답.

In [1]:
# !pip install chromadb tiktoken transformers sentence_transformers openai langchain pypdf

### Retrieval : 검색을 쉽게할 수 있게 하는 모듈

In [2]:
from langchain_openai import OpenAIEmbeddings
import json

with open("api_keys.json") as f:
    OPENAI_API_KEY = json.load(f)["openai"]

embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [3]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base") # openai 의 토크나이저

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [8]:
from langchain.chains import RetrievalQA # retrieval 모듈을 이용한 QA
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

In [9]:
loader = PyPDFLoader("docs/doc_1.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10,
                                               chunk_overlap=5,
                                               length_function=tiktoken_len)
                                      
docs = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf_embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

docsearch = Chroma.from_documents(docs, hf_embedding_model) # 임베딩 과정

In [10]:
docsearch.similarity_search("which place on fire?")

[Document(metadata={'page': 0, 'source': 'docs/doc_1.pdf'}, page_content='Tokyo is on fire'),
 Document(metadata={'page': 1, 'source': 'docs/doc_1.pdf'}, page_content='Tokyo is on fire'),
 Document(metadata={'page': 0, 'source': 'docs/doc_1.pdf'}, page_content='Tokyo is on fire'),
 Document(metadata={'page': 1, 'source': 'docs/doc_1.pdf'}, page_content='Tokyo is on fire')]

In [40]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

gpt4o = ChatOpenAI(model_name="gpt-4o-mini",
                   streaming=True,
                   callbacks=[StreamingStdOutCallbackHandler()],
                   temperature=0.5, # 일관된 답을 위해
                   max_tokens=1000,
                   openai_api_key=OPENAI_API_KEY)

qa = RetrievalQA.from_chain_type(
    llm=gpt4o,
    chain_type="stuff",
    retriever=docsearch.as_retriever( # 
        search_type = "mmr", # 연관성 높은 풀 여러개 중 다양하게 조합해서 llm 에게 넘겨준다.
        search_kwargs={'k':5, 'fetch_k':1}  # 50 개 중에 10 개
        ),
    return_source_documents = True)

query = "which place is on fire?"
result = qa(query)

Tokyo is on fire.

In [ ]:
result

{'query': 'which place is on fire?',
 'result': 'Tokyo is on fire.',
 'source_documents': [Document(metadata={'page': 0, 'source': 'docs/doc_1.pdf'}, page_content='Tokyo is on fire')]}